https://westus.dev.cognitive.microsoft.com/docs/services/56f91f2d778daf23d8ec6739/operations/587f2c6a154055056008f200

https://nordicapis.com/digitize-your-notes-with-microsoft-vision-api/

https://stackoverflow.com/questions/44126333/python-method-to-upload-an-image-to-microsoft-cognitive-vision-from-local-machin

In [1]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, requests, time, json  
import os
import re
from shutil import copy
from dotenv import load_dotenv

In [9]:
#import environment variables
load_dotenv()
ENDPOINT = os.getenv('ENDPOINT')
API_KEY = os.getenv('API_KEY')

In [3]:
params = {'handwriting' : 'true'}

### when request via uploaded file

In [4]:
# when request via url
headers = {  
    # Request headers.  
    # Another valid content type is "application/octet-stream".  
    'Content-Type': 'application/octet-stream',  
    'Ocp-Apim-Subscription-Key': api_key,  
}


In [5]:
dirpath_input = '../../data/source/'
dirpath_output = '../../data/converted/'

In [6]:
def write_to_textfile(dirpath_output, lines, filename):
    """
    Write recognitionResult to text file 
    """
    image_converted = open("{0}{1}.txt".format(dirpath_output,filename),"w+")
    for line in lines:
        text=line['text']
        image_converted.write(text+ '\n')
    image_converted.close()

In [7]:
for i in range(len(os.listdir(dirpath_input))):
    # get image_filename with and without extension
    img_filename = os.listdir(dirpath_input)[i]
    filename = re.split(r"\b\.", img_filename)[0]
    
    #read image
    with open(dirpath_input+img_filename, 'rb') as f:
        img_data = f.read()
    
    # request from uploaded file
    response = requests.request('POST',
                                endpoint+ '/RecognizeText',
                                data=img_data,
                                headers=headers,
                                params=params)
    
    # Display JSON data and exit if the REST API call was not successful.
    if response.status_code != 202:
        parsed = json.loads(response.text)  
        print ("Error at iteration {0}, filename {1}:".format(i,img_filename))  
        print (json.dumps(parsed, sort_keys=True, indent=2))  
        exit()
        
    # grab the 'Operation-Location' from the response
    operationLocation = response.headers['Operation-Location']
    
    # Waiting 10 seconds to retrieve the recognized text')  
    time.sleep(10)
    
    # Execute the second REST API call and get the response.
    response = requests.request('GET', 
                                operationLocation, 
                                json=None, 
                                data=None, 
                                headers=headers, 
                                params=None)
     
    # The following formats the JSON data for display.  
    parsed = json.loads(response.text)
    lines = parsed['recognitionResult']['lines']
    
    # new folder name 
    new_folder = dirpath_output+filename+'/'
    
    # if folder already exists delete folder and files
    if os.path.exists(new_folder):
        os.rmdir(new_folder)
    # make new folder
    os.makedirs(new_folder)

    # write results to textfile in new folder
    write_to_textfile(new_folder, lines, filename)

    # copy image to new_folder
    copy(dirpath_input+img_filename, new_folder)
    


Error at iteration 29, filename 00000824-VIDEO-2017-08-12-05-57-28.mp4:
{
  "error": {
    "code": "InvalidImage",
    "message": "Input data is not a valid image."
  }
}


KeyError: 'operation-location'